In [32]:
import importlib
import pandas as pd
import numpy as np
from OT2_code import OT2Directions as OT2D
from OT2_code import Create_Directions as CD
import opentrons.simulate as simulate
from datetime import datetime
from pytz import timezone 

## Import values for Volume, Time, and Order
The three arrays contain information on what to perform. 
- **v_array**: This is the volume array and it contains information on the volumes of each stock solution, in microliters, used to make one sample. Each column represents a stock solution and each row represents a sample. 
- **o_array**: This is the order in which the stock solutions are added. Each row represents a sample and each column represents the next stock solution that should be added. The number 1 represents the first stock solution that should be added. 
- **t_array**: This is the time array and it contains information on the delay times before adding each stock solution. The first colum represents the time delay that occurs before adding the second stock solution to the first. Note that the delay times are applied to the stock solutions of the samples after they are ordered according to the values in the o_array. The units of the time delay are in units of the **action_time** which is the amount of time it takes in seconds to perform one action. More information on this can be found in the next section. 

In [33]:
path = 'Samples_and_Protocol/Samples/'
v_array = pd.read_csv(path + 'volume.csv').values
t_array = pd.read_csv(path + 'time.csv').values
o_array = pd.read_csv(path + 'order.csv').values
v_cols = pd.read_csv(path + 'volume.csv').columns
t_cols = pd.read_csv(path + 'time.csv').columns
o_cols = pd.read_csv(path + 'order.csv').columns

## Create the Direction Array
The function CD.create_directions will transform the volume, time, and order arrays into a single array with the directions that the OT2 can follow. The OT2 will perform one action (pick up tip - transfer stock to destination - drop tip) per row and every action will take the same amount of time which can be set by the keyword argument **action_time**. The first column of the direction_array variable contains information on the volume that needs to be transferred. The second column contains information on the sample number. The third column contains information on the stock number to use. A row that contains negative numbers means that the OT2 will not perform any action for a duration equal to the absolute value of the negative number multiplied by the time it takes to perform one action. Any sample with a volume = 0 and time_delay = 0, will be completely ignored. 

In [34]:
direction_array = CD.create_directions(v_array, t_array, o_array)
exp = OT2D.experiment()
total_time = exp.calculate_exp_duration(direction_array, action_time = 60)

Experiment will take a total time of:
1 hours
3 minutes


### Optional: Optimize the total time taken
Running the following code will minimize the total experimental time taken by shuffling the sample locations. This will reshuffle the sample locations in v_array, t_array, and o_array. The drawback is that it may take a long time to perform the calculation. Remember to resave the new arrays.

In [35]:
exp = OT2D.experiment()
order_times, best_time = exp.optimize(v_array, t_array, o_array)
direction_array, v_array, t_array, o_array = exp.change_order(order_times, v_array, t_array, o_array)
total_time = exp.calculate_exp_duration(direction_array, action_time = 60)

Experiment will take a total time of:
0 hours
45 minutes


Save the DataFrames

In [36]:
volume = pd.DataFrame(v_array, columns = v_cols)
time = pd.DataFrame(t_array, columns = t_cols)
order = pd.DataFrame(o_array, columns = o_cols)

## Determine the Experiment Duration
Specify the protocol path. This determines the identiy and locations of the labwares on the OT2. 

In [37]:
protocol_path = 'Samples_and_Protocol/Protocol/protocol.csv'

Set the time that each action should take with the keyword argument **action_time**, which has units of seconds. 

In [39]:
exp = OT2D.experiment()
protocol = exp.simulate(protocol_path)
_ = exp.calculate_exp_duration(direction_array, action_time = 60)

/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults
/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults


dest offset loaded
stock offset loaded
Experiment will take a total time of:
0 hours
45 minutes


## Start the OT2
The following line of code will determine where to place the stock solutions in the labware specified in the file *protocol.csv*. The variable **vial_volume** is the maximum volume of each stock solution vial in mircoliters. It is assumed that every stock solution vial has the same volume.  

In [40]:
vial_volume = 17000
direction_array = exp.stock_solution_ranges(direction_array, vial_volume)

Stock 0 on  A1 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 1 on  A2 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 2 on  A3 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 3 on  A4 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 4 on  B1 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 5 on  B2 of 20mLscintillation 12 Well Plate 18000 µL on 1
Stock 6 on  B3 of 20mLscintillation 12 Well Plate 18000 µL on 1


Determine which well in the wellplate to start in

In [41]:
start_location = 0

The following line of code will simulate the OT2 commands used to perform the experiment. 

In [42]:
protocol = exp.simulate(protocol_path)
exp.perform_directions(protocol, direction_array, start_location = start_location)

/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults
/home/huatthart/.opentrons/robot_settings.json not found. Loading defaults
/home/huatthart/.opentrons/deck_calibration.json not found. Loading defaults


dest offset loaded
stock offset loaded
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 112.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 500.0 uL/sec
Air gap
Aspirating 20.0 uL from A1 of 20mLscintillation 12 Well Plate 18000 µL on 1 at 500.0 uL/sec
Dispensing 112.0 uL into A1 of 7 on Temperature Module GEN2 on 7 at 1050.0 uL/sec
Blowing out at A1 of 7 on Temperature Module GEN2 on 7
Mixing 2 times with a volume of 100.0 ul
Aspirating 100.0 uL from A1 of 7 on Temperature Module GEN2 on 7 at 750.0 uL/sec
Dispensing 100.0 uL into A1 of 7 on Temperature Module GEN2 on 7 at 1050.0 uL/sec
Aspirating 100.0 uL from A1 of 7 on Temperature Module GEN2 on 7 at 750.0 uL/sec
Dispensing 100.0 uL into A1 of 7 on Temperature Module GEN2 on 7 at 1050.0 uL/sec
Mixing 2 times with a volume of 150.0 ul
Aspirating 150.0 uL from A6 of Agilent 6 Reservoir 47000 µL on 5 at 1000.0 uL/sec
Dispensing 150.0 uL into A6 of Agilent 6 Reservoir 47000 µL on 5 at 1400.0 uL/sec

## Transfer Samples from one Labware to Another

In [43]:
# well_index = np.array([1,5,3,7,2,8,4,10,23])
# protocol = exp.simulate(protocol_path)
# exp.transfer_samples(protocol, 20, 1, source_well_index = well_index, starting_position = 5)

## Run the Experiment

In [9]:
Pacific = timezone('US/Pacific')
p_time = datetime.now(Pacific)
print(p_time.strftime('Start Time: %H:%M:%S'))

Start Time: 22:18:21


In [ ]:
protocol = exp.execute(protocol_path)
exp.perform_directions(protocol, direction_array, start_location = start_location)

In [ ]:
Pacific = timezone('US/Pacific')
p_time = datetime.now(Pacific)
print(p_time.strftime('End Time: %H:%M:%S'))

## Test Accuracy of Experiment

In [9]:
exp.test(exp.exp_data, v_array, t_array, o_array)

Time: The average time delay error is (+/-) 120 seconds
Order: Check Passed, Actual Order is equal to specified Order
Volume: Check Passed, Actual Volume is equal to specified Volume
